# Fisher Scoring

In [1]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def get_coefficients(design_matrix, response_vector, epsilon=.001):
    """
    Determine Logistic Regression coefficents using Fisher Scoring algorithm.
    Iteration ceases once changes between elements in coefficent matrix across
    consecutive iterations is less than epsilon.
    # =========================================================================
    # design_matrix      `X`     => n-by-(p+1)                                |
    # response_vector    `y`     => n-by-1                                    |
    # probability_vector `p`     => n-by-1                                    |
    # weights_matrix     `W`     => n-by-n                                    |
    # epsilon                    => threshold above which iteration continues |
    # =========================================================================
    # n                          => # of observations                         |
    # (p + 1)                    => # of parameterss, +1 for intercept term   |
    # =========================================================================
    # U => First derivative of Log-Likelihood with respect to                 |
    #      each beta_i, i.e. `Score Function`: X_transpose * (y - p)          |
    #                                                                         |
    # I => Second derivative of Log-Likelihood with respect to                |
    #      each beta_i. The `Information Matrix`: (X_transpose * W * X)       |
    #                                                                         |
    # X^T*W*X results in a (p+1)-by-(p+1) matrix                              |
    # X^T(y - p) results in a (p+1)-by-1 matrix                               |
    # (X^T*W*X)^-1 * X^T(y - p) results in a (p+1)-by-1 matrix                |
    # ========================================================================|
    """
    X = np.matrix(design_matrix)
    y = np.matrix(response_vector)

    # initialize logistic function used for Scoring calculations =>
    def pi_i(v): return (np.exp(v) / (1 + np.exp(v)))

    # initialize beta_0, p_0, W_0, I_0 & U_0 =>
    beta_0 = np.matrix(np.zeros(np.shape(X)[1])).T
    p_0 = pi_i(X * beta_0)
    W_pre = (np.array(p_0) * np.array(1 - p_0))
    W_0 = np.matrix(np.diag(W_pre[:, 0]))
    I_0 = X.T * W_0 * X
    U_0 = X.T * (y - p_0)

    # initialize variables for iteration =>
    beta_old = beta_0
    iter_I = I_0
    iter_U = U_0
    iter_p = p_0
    iter_W = W_0
    fisher_scoring_iterations = 0
    coeffs = []

    # iterate until abs(beta_new - beta_old) < epsilon =>
    while True:

        # Fisher Scoring Update Step =>
        coeffs.append(np.array(beta_old))
        fisher_scoring_iterations += 1
        beta_new = beta_old + iter_I.I * iter_U

        if all(np.abs(np.array(beta_new)-np.array(beta_old)) < epsilon):
            model_parameters  = beta_new
            fitted_values     = pi_i(X * model_parameters)
            covariance_matrix = iter_I.I
            break

        else:
            iter_p     = pi_i(X * beta_new)
            iter_W_pre = (np.array(iter_p) * np.array(1 - iter_p))
            iter_W     = np.matrix(np.diag(iter_W_pre[:, 0]))
            iter_I     = X.T * iter_W * X
            iter_U     = X.T * (y - iter_p)
            beta_old   = beta_new

    summary = {
        'model_parameters' : np.array(model_parameters),
        'fitted_values'    : np.array(fitted_values),
        'covariance_matrix': np.array(covariance_matrix),
        'number_iterations': fisher_scoring_iterations
    }

    return (summary)

Leemos en el conjunto de datos y lo particionamos en la matriz de diseño y el vector de respuesta, que luego se pasan a get_coefficients:

In [15]:
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

# read dataset with pandas =>
df = pd.read_csv('hcc-data-complete-balanced.csv') #cargamos tabla

# add intercept field =>
df['INTERCEPT'] = 1


In [16]:
np.isnan(df).any()

np.isinf(df).any()

Gender            False
Symptoms          False
Alcohol           False
HBsAg             False
HBeAg             False
HBcAb             False
HCVAb             False
Cirrhosis         False
Endemic           False
Smoking           False
Diabetes          False
Obesity           False
Hemochro          False
AHT               False
CRI               False
HIV               False
NASH              False
Varices           False
Spleno            False
PHT               False
PVT               False
Metastasis        False
Hallmark          False
Age               False
Grams_day         False
Packs_year        False
PS                False
Encephalopathy    False
Ascites           False
INR               False
AFP               False
Hemoglobin        False
MCV               False
Leucocytes        False
Platelets         False
Albumin           False
Total_Bil         False
ALT               False
AST               False
GGT               False
ALP               False
TP              

In [19]:
#variables = df.drop(['Class'], axis=1)
#X = variables.values

'''X = df[['INTERCEPT','Gender','Symptoms','Alcohol','HBsAg','HBeAg','HBcAb','HCVAb','Cirrhosis','Endemic','Smoking','Diabetes',
        'Obesity','Hemochro','AHT','CRI','HIV','NASH','Varices','Spleno','PHT','PVT','Metastasis','Hallmark','Age','Grams_day',
        'Packs_year','PS','Encephalopathy','Ascites','INR','AFP','Hemoglobin','MCV','Leucocytes','Platelets','Albumin',
        'Total_Bil','ALT','AST','GGT','ALP','TP','Creatinine','Nodule','Major_Dim','Dir_Bil','Iron','Sat','Ferritin']].values'''

X = df[['INTERCEPT','Metastasis']].values
y = df[['Class']].values

# call `get_coefficients`, keeping epsilon at .001 =>
my_summary = get_coefficients(X, y, epsilon=.001)

get_coefficients devuelve un diccionario con los siguientes pares clave-valor:

    'model_parameters': Los coefficentes estimados del modelo
    'covariance_matrix': La matriz de varianza-covarianza de las estimaciones coefficentes
    'fitted_values': Las variables explicativas ajustadas
    'number_iterations': el número de iteraciones de Fisher Scoring

In [20]:
my_summary['model_parameters']

array([[ 0.30294954],
       [-1.27481013]])

In [21]:
my_summary['covariance_matrix']

array([[ 0.02674825, -0.02674825],
       [-0.02674825,  0.12520384]])

In [22]:
my_summary['fitted_values']

array([[0.5751634],
       [0.5751634],
       [0.2745098],
       [0.2745098],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.2745098],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.2745098],
       [0.2745098],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.2745098],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.2745098],
       [0.5751634],
       [0.5751634],
       [0.2745098],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.5751634],
       [0.2745098],
       [0.5751634],
       [0.5751634],


para la variable 'Metastasis' se obtiene un **coeficiente** de -1,2748 

El **vector de valores ajustados** correspondientes a los datos de la variable. Es decir, que para cada punto de la muestra obtenemos un valor predicho de la probabilidad: 0,57516 en caso de No tener metastasis y 0,2745 cuando hay metástasis.

In [23]:
my_summary['number_iterations']

4

Los coefficentes negativos corresponden a variables que están correlacionadas negativamente con la probabilidad de un resultado positivo ('Class'=1, el paciente sobrevive).


 